In [1]:
!pip install tqdm

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
import pandas as pd
import numpy as np
# from tqdm import tqdm

from scipy import stats

print(pp.__version__)



2.0.1


In [3]:
def create_measurement_unit(df_measurement, net):
    list_value=[]
    list_std=[]
#     print(net)
    for index, row in df_measurement.iterrows():
        if row['element_type'] =='bus':
            if row['meas_type'] =='v':
                mu= net.res_bus.iloc[row['element'],0]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
            elif row['meas_type'] =='p':
                mu= net.res_bus.iloc[row['element'],2]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
            elif row['meas_type'] =='q':
                mu= net.res_bus.iloc[row['element'],3]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4

        elif row['element_type'] =='line':

            if row['side']=='from':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],0]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],1]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],6]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['side']=='to':
                pass

        elif row['element_type'] =='trafo':
            'create trafo measurement'
            if row['side']=='from':
                pass
            elif row['side']=='to':
                if row['meas_type'] =='v':
                    mu =net.res_trafo.iloc[row['element'],10]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        #                 print(mu)
                elif row['meas_type'] =='p':
                    mu =net.res_trafo.iloc[row['element'],2]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type'] =='q':
                    mu =net.res_trafo.iloc[row['element'],3]
        #                 print(mu)
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        #     print('Mean:',mu, 'standard deviation:', sigma)
        value = np.random.normal(mu, sigma, 1)
        #     print('Random value:', value[0])
        list_value.append(value[0])
        list_std.append(sigma)
#     print(list_value)
    df_measurement['value']=list_value
    df_measurement['std_dev']=list_std

    for index, row in df_measurement.iterrows():
        if row['element_type']=='bus':
#             print('create bus measurement')
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'])

        elif row['element_type']=='line':
#             print('create line measurement')
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])

        elif row['element_type']=='trafo':
#             print('create trafo measurement')
            if row['meas_type'] in ['p','q']:
                pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                      std_dev=row['std_dev'], element=row['element'], side=row['side'])
    return df_measurement, net

# NO solar PV

In [73]:
net1 = pn.simple_four_bus_system()
net1.sgen['p_mw']=[0,0]
net1.sgen['q_mvar']=[0,0]
net1.sgen['sn_mva']=[0,0]
pp.runpp(net1)

In [74]:
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']
df_measurement, net1 = create_measurement_unit(df_measurement, net1)
success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net1.res_bus_est.vm_pu, net1.res_bus_est.va_degree

In [75]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=[0]
participation_bus=list(set(list(net1.bus.index))-set(list_slack_bus+list_zero_inject_bus))

In [76]:
df_LPF=pd.DataFrame()
df_LPF['name']=net1.bus['name'].copy()
df_LPF['name']=net1.bus['name'].copy()
known_generation=-net1.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net1.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net1.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net1.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net1.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: 10.569592562200846 %


In [77]:
net1

This pandapower network includes the following parameter tables:
   - bus (4 elements)
   - load (2 elements)
   - sgen (2 elements)
   - ext_grid (1 element)
   - line (2 elements)
   - trafo (1 element)
   - measurement (10 elements)
   - bus_geodata (4 elements)
 and the following results tables:
   - res_bus (4 elements)
   - res_line (2 elements)
   - res_trafo (1 element)
   - res_ext_grid (1 element)
   - res_load (2 elements)
   - res_sgen (2 elements)
   - res_bus_power_flow (4 elements)
   - res_ext_grid_power_flow (1 element)
   - res_line_power_flow (2 elements)
   - res_load_power_flow (2 elements)
   - res_sgen_power_flow (2 elements)
   - res_trafo_power_flow (1 element)
   - res_bus_est (4 elements)
   - res_line_est (2 elements)
   - res_trafo_est (1 element)

In [78]:
net1.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.078473,0.021935,-0.065420,-0.020252,0.013053,0.001683,0.116424,0.116426,0.116426,1.010171,-150.611715,0.849008,-149.030927,81.990442
1,0.033864,0.010297,-0.030377,-0.009850,0.003487,0.000447,0.060174,0.060176,0.060176,0.849008,-149.030927,0.765967,-147.984220,42.377638


In [80]:
net1.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,1.005372,0.000000,-0.056013,0.048746
1,1.010171,-150.611715,-0.023330,-0.071509
2,0.849008,-149.030927,0.031556,0.009955
3,0.765967,-147.984220,0.030377,0.009850


In [81]:
line_loss

0.016540452159173366

In [82]:
line_loss_participation

0.29529826360477

In [83]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,-0.416519
2,bus3,0.031556,0.009955,0.563371
3,bus4,0.030377,0.009850,0.542325


In [171]:
list_LPF_zero_injection_bus=[]
for round in range(0,100):
# for round in tqdm(range(0,100)):
#     print(round)
    net1 = pn.simple_four_bus_system()
    net1.sgen['p_mw']=[0,0]
    net1.sgen['q_mvar']=[0,0]
    net1.sgen['sn_mva']=[0,0]
    pp.runpp(net1)
    df_measurement=pd.DataFrame()
    df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
    df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
    df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
    df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']
    df_measurement, net1 = create_measurement_unit(df_measurement, net1)
    success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
#     V_est, delta_est = net1.res_bus_est.vm_pu, net1.res_bus_est.va_degree
    list_slack_bus=[0]
    list_zero_inject_bus=[1]
    list_known_generation_bus=[0]
    participation_bus=list(set(list(net1.bus.index))-set(list_slack_bus+list_zero_inject_bus))
    df_LPF=pd.DataFrame()
    df_LPF['name']=net1.bus['name'].copy()
    known_generation=-net1.res_bus_est.iloc[list_slack_bus[0],2]

    allocate_p_MW=[]
    allocate_q_MVAR=[]
    for index, row in df_LPF.iterrows():
        if index in list_known_generation_bus+list_zero_inject_bus:
            allocate_p_MW.append(0)
            allocate_q_MVAR.append(0)
        else:
            allocate_p_MW.append(net1.res_bus_est.iloc[index,2])
            allocate_q_MVAR.append(net1.res_bus_est.iloc[index,3])
    df_LPF['allocate_p_MW']=allocate_p_MW
    df_LPF['allocate_1_MVAR']=allocate_q_MVAR


    df_LPF['LPF']=net1.res_bus_est['p_mw']/known_generation

    loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

#     print('loss is:', loss*100, '%')
    list_LPF_zero_injection_bus.append(df_LPF.LPF[1])

In [172]:
stats.trim_mean(np.array(list_LPF_zero_injection_bus), 0.1)

-0.09138461488661316

In [173]:
np.array(list_LPF_zero_injection_bus).mean()

0.34064070238724187

# integration solar PV

In [66]:
net2 = pn.simple_four_bus_system()
pp.runpp(net2)
V, delta=net2.res_bus.vm_pu, net2.res_bus.va_degree
# print(net)

In [67]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net2 = create_measurement_unit(df_measurement, net2)

success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree


In [68]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net2.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

In [69]:
df_LPF=pd.DataFrame()
df_LPF['name']=net2.bus['name'].copy()
df_LPF['name']=net2.bus['name'].copy()
known_generation=-net2.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net2.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net2.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net2.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net2.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -342.5323174233025 %


In [70]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,3.787364
2,bus3,0.011675,0.005126,-1.066840
3,bus4,0.014867,0.008017,-1.358483


In [71]:
net2.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.029256,0.013485,-0.027224,-0.013227,0.002033,0.000258,0.045940,0.045943,0.045943,1.012140,-150.215436,0.950887,-149.056207,32.354131
1,0.015549,0.008101,-0.014867,-0.008017,0.000682,0.000084,0.026613,0.026616,0.026616,0.950887,-149.056207,0.915961,-148.252009,18.743916


In [72]:
net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.994000,0.000000,0.010944,0.114551
1,1.012140,-150.215436,-0.041447,-0.130082
2,0.950887,-149.056207,0.011675,0.005126
3,0.915961,-148.252009,0.014867,0.008017


In [52]:
line_loss_participation

0.0992062083797662

In [174]:
list_LPF_zero_injection_bus=[]
for round in range(0,100):
# for round in tqdm(range(0,100)):
#     print(round)
    net2 = pn.simple_four_bus_system()
    pp.runpp(net2)
    V, delta=net2.res_bus.vm_pu, net2.res_bus.va_degree
    # print(net)
    df_measurement=pd.DataFrame()
    df_measurement=pd.DataFrame()
    df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
    df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
    df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
    df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

    #where bus measurement has 3%error
    #where line measurement has 5%error
    #where trafo measurement has 5%error

    df_measurement, net2 = create_measurement_unit(df_measurement, net2)

    success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])
    V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

    list_slack_bus=[0]
    list_zero_inject_bus=[1]
    list_known_generation_bus=list(set(list_slack_bus))
    participation_bus=list(set(list(net2.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))
    df_LPF=pd.DataFrame()
    df_LPF['name']=net2.bus['name'].copy()
    known_generation=-net2.res_bus_est.iloc[list_slack_bus[0],2]

    allocate_p_MW=[]
    allocate_q_MVAR=[]
    for index, row in df_LPF.iterrows():
        if index in list_known_generation_bus+list_zero_inject_bus:
            allocate_p_MW.append(0)
            allocate_q_MVAR.append(0)
        else:
            allocate_p_MW.append(net2.res_bus_est.iloc[index,2])
            allocate_q_MVAR.append(net2.res_bus_est.iloc[index,3])
    df_LPF['allocate_p_MW']=allocate_p_MW
    df_LPF['allocate_q_MVAR']=allocate_q_MVAR


    df_LPF['LPF']=net2.res_bus_est['p_mw']/known_generation

    loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

    # print('loss is:', loss*100, '%')
    list_LPF_zero_injection_bus.append(df_LPF.LPF[1])

In [175]:
stats.trim_mean(np.array(list_LPF_zero_injection_bus), 0.1)

0.23873394210645643

In [176]:
df_LPF

,name,allocate_p_MW,allocate_q_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,6.851735
2,bus3,0.010696,0.005426,-2.104078
3,bus4,0.015251,0.008021,-2.999953


# Solar PV at bus 2

In [100]:
net3 = pn.simple_four_bus_system()
net3.sgen['p_mw']=[0.020,0]
net3.sgen['q_mvar']=[0.005,0]
net3.sgen['sn_mva']=[0.03,0]
pp.runpp(net3)

In [101]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net3 = create_measurement_unit(df_measurement, net3)

success = estimate(net3, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

In [102]:
success

True

In [103]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net3.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

df_LPF=pd.DataFrame()
df_LPF['name']=net3.bus['name'].copy()
known_generation=-net3.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net3.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net3.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net3.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net3.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -58.64858392198169 %


In [104]:
print(df_LPF)

      name  allocate_p_MW  allocate_1_MVAR       LPF
0  bus1ref       0.000000         0.000000 -1.000000
1     bus2       0.000000         0.000000  0.476582
2     bus3       0.012648         0.005001  0.124887
3     bus4       0.029230         0.010303  0.288627


In [105]:
net3.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.050573,0.016420,-0.044813,-0.015680,0.005760,0.000740,0.077339,0.077341,0.077341,0.992350,-150.403840,0.886039,-149.111462,54.465588
1,0.032165,0.010679,-0.029230,-0.010303,0.002936,0.000376,0.055210,0.055213,0.055213,0.886039,-149.111462,0.810215,-148.066953,38.882058


In [106]:
known_generation

0.1012720125134151

In [107]:
net3.res_bus_est.iloc[list_slack_bus[0],2]

-0.1012720125134151

In [108]:
net3.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,1.022733,0.000000,-0.101272,-0.171998
1,0.992350,-150.403840,0.048264,0.149774
2,0.886039,-149.111462,0.012648,0.005001
3,0.810215,-148.066953,0.029230,0.010303


In [109]:
sum(net3.res_line_est['pl_mw'])

0.008695686058934258

In [110]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,0.476582
2,bus3,0.012648,0.005001,0.124887
3,bus4,0.029230,0.010303,0.288627


# Solar PV panel at bus 3

In [127]:
net4 = pn.simple_four_bus_system()
# print(net4.sgen)
net4.sgen['p_mw']=[0,0.015]
net4.sgen['q_mvar']=[0,0.002]
net4.sgen['sn_mva']=[0,0.02]
print(net4.sgen)

pp.runpp(net4)

  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.000   0.000    0.00      1.0        True  None            True
1  pv2    3  0.015   0.002    0.02      1.0        True  None            True


In [128]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net4 = create_measurement_unit(df_measurement, net4)

success = estimate(net4, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

In [129]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net3.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

df_LPF=pd.DataFrame()
df_LPF['name']=net4.bus['name'].copy()
known_generation=-net4.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net4.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net4.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net4.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net4.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -35.83392490812809 %


In [130]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,0.243407
2,bus3,0.030431,0.009766,0.428705
3,bus4,0.015116,0.007486,0.212956


In [132]:
net4.res_line_est


,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.052614,0.018157,-0.046343,-0.017351,0.006271,0.000806,0.080696,0.080699,0.080699,0.995544,-150.414255,0.885079,-148.932422,56.830152
1,0.015912,0.007585,-0.015116,-0.007486,0.000796,0.000099,0.028747,0.028750,0.028750,0.885079,-148.932422,0.846880,-148.084316,20.246275
